### Import

In [13]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder


### Data Load

In [14]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [15]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [16]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [17]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [18]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [19]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [20]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

### Train

In [21]:
import torch

print("Is CUDA available? :", torch.cuda.is_available())
print(torch.__version__)

Is CUDA available? : False
2.5.1


TabNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

# ✅ GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ 데이터 분할 (훈련 / 검증)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_encoded.values, y.values, test_size=0.2, random_state=42
)

# ✅ 모델 입력 차원 설정
input_dim = X_train.shape[1]  # 실제 데이터 차원 자동 설정
n_heads = 8
hidden_dim = (max(64, input_dim) // n_heads) * n_heads  # `n_heads`의 배수로 자동 조정

# ✅ PyTorch Tensor 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)  # (N, 1) 형태로 변경
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32).to(device)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32).unsqueeze(1).to(device)

# ✅ PyTorch DataLoader 생성 (미니배치 학습 가능)
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# ✅ TabTransformer 모델 정의
class TabTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_heads=8, n_layers=2):
        super(TabTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)  # 입력 차원을 임베딩 차원으로 변환
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=n_heads, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)
        self.fc = nn.Linear(hidden_dim, 1)  # 최종 예측 층 (이진 분류)

    def forward(self, x):
        x = self.embedding(x)  # 입력 데이터 변환
        x = x.unsqueeze(1)  # Transformer 입력 차원 맞추기 (batch, seq_len, hidden_dim)
        x = self.transformer_encoder(x)
        x = self.fc(x[:, 0, :])  # 첫 번째 토큰을 예측에 사용
        return x  # Sigmoid 제거 (Loss 계산할 때 적용)

# ✅ 모델 생성
model = TabTransformer(input_dim=input_dim, hidden_dim=hidden_dim, n_heads=n_heads).to(device)
criterion = nn.BCEWithLogitsLoss()  # Sigmoid 포함된 Loss 함수 사용
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ 모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()  # (32, 1) → (32,) 변환
        loss = criterion(y_pred, y_batch.squeeze())  # y_batch도 차원 맞추기!
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # ✅ 검증
    model.eval()
    valid_loss = 0.0
    y_valid_preds = []

    with torch.no_grad():
        for X_batch, y_batch in valid_loader:
            y_valid_pred = model(X_batch).squeeze()
            loss = criterion(y_valid_pred, y_batch.squeeze())
            valid_loss += loss.item()
            y_valid_preds.extend(torch.sigmoid(y_valid_pred).cpu().numpy())  # 여기서 sigmoid 적용

    y_valid_preds = np.array(y_valid_preds)
    valid_acc = accuracy_score(y_valid, y_valid_preds.round())
    valid_auc = roc_auc_score(y_valid, y_valid_preds)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss/len(train_loader):.4f}, "
          f"Valid Loss: {valid_loss/len(valid_loader):.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}, Valid ROC-AUC: {valid_auc:.4f}")


Using device: cpu
Epoch 1/100 - Train Loss: 0.5108, Valid Loss: 0.4973, Valid Accuracy: 0.7440, Valid ROC-AUC: 0.7256
Epoch 2/100 - Train Loss: 0.5010, Valid Loss: 0.4972, Valid Accuracy: 0.7440, Valid ROC-AUC: 0.7262
Epoch 3/100 - Train Loss: 0.4984, Valid Loss: 0.4926, Valid Accuracy: 0.7435, Valid ROC-AUC: 0.7295
Epoch 4/100 - Train Loss: 0.4969, Valid Loss: 0.4936, Valid Accuracy: 0.7443, Valid ROC-AUC: 0.7311
Epoch 5/100 - Train Loss: 0.4963, Valid Loss: 0.4945, Valid Accuracy: 0.7440, Valid ROC-AUC: 0.7285
Epoch 6/100 - Train Loss: 0.4955, Valid Loss: 0.4906, Valid Accuracy: 0.7459, Valid ROC-AUC: 0.7322
Epoch 7/100 - Train Loss: 0.4953, Valid Loss: 0.4926, Valid Accuracy: 0.7459, Valid ROC-AUC: 0.7314
Epoch 8/100 - Train Loss: 0.4949, Valid Loss: 0.4903, Valid Accuracy: 0.7464, Valid ROC-AUC: 0.7326
Epoch 9/100 - Train Loss: 0.4947, Valid Loss: 0.4895, Valid Accuracy: 0.7447, Valid ROC-AUC: 0.7335
Epoch 10/100 - Train Loss: 0.4939, Valid Loss: 0.4938, Valid Accuracy: 0.7449, Val

In [23]:
torch.save(model.state_dict(), "./tabtransformer_model.pth")

### Predict

In [24]:
import torch

def predict_proba(model, X):
    model.eval()  # 평가 모드로 전환
    X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32).to(device)  # ✅ NumPy 배열로 변환 후 Tensor 변환
    with torch.no_grad():
        logits = model(X_tensor).squeeze()  # 모델 예측값
        probabilities = torch.sigmoid(logits)  # Sigmoid 적용하여 확률로 변환
    return probabilities.cpu().numpy()


In [25]:
pred_proba = predict_proba(model, X_test_encoded)  # 확률 예측

### Submission

In [26]:
sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [27]:
sample_submission.to_csv('./TabTransformer_submit.csv', index=False)